<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'>&copy; 2015 Karim Pichara - Christian Pieringer. Todos los derechos reservados. Modificado en 2018-1 por Equipo Docente IIC2233.</font>
</p>

## Funciones _lambda_

Antes de explicar qué son las funciones _lambda_, necesitamos hablar sobre cómo se tratan las funciones en Python. En el caso de Python, se dice que el lenguaje tiene funciones de primera clase (_first-class functions_), es decir, que las funciones son tratadas como cualquier otra variable. Esto en otros lenguajes como Java se ha empezado a incorporar de a poco con ciertas limitaciones.

El que las funciones sean de primera clase tiene algunas consecuencias en las que profundizaremos la próxima semana, como:

1\. Las funciones pueden ser asignadas a una variable, y luego usar esa variable igual que la función.


In [1]:
def suma(x, y):
    return x + y

adición = suma

print(suma(3, 5))
print(adición(3, 5))

8
8


2\. Las funciones pueden ser pasadas como parámetro a otras funciones.

In [2]:
def saludar_señora(nombre):
    return ' '.join(["Señora", nombre])

def saludar_señor(nombre):
    return ' '.join(["Señor", nombre])

def saludar_tarde(función_saludo, nombre):
    return ' '.join(["Buenas tardes", función_saludo(nombre)])

print(saludar_tarde(saludar_señora, "Valeria"))
print(saludar_tarde(saludar_señor, "Germán"))

Buenas tardes Señora Valeria
Buenas tardes Señor Germán


**Las funciones _lambda_** son una forma alternativa de definir funciones en Python. Además de su nombre griego, no hay nada intimidante en ellas. Veamos un ejemplo de cómo definirlas:

In [3]:
sucesor = lambda x: x + 1

# Es (casi) equivalente a

def sumar_uno(x):
    return x + 1

In [4]:
restar = lambda x, y: x - y

# Es (casi) equivalente a
def sustracción(x, y):
    return x - y

Como se puede observar, la sintaxis consiste en `lambda <parámetros>: <valor a retornar>`. En estas funciones no se necesita la sentencia `return`, puesto que la operación que se coloca a la derecha de los dos puntos (`:`) es el valor que se devolverá.

Una característica que distingue a las funciones _lambda_ es que **pueden ser definidas en forma anónima**, es decir, funciones que no reciben un nombre específico.

In [5]:
restar.__name__

'<lambda>'

In [6]:
sustracción.__name__

'sustracción'

Estas funciones pueden ser vistas como _fugaces_ y son utilizadas únicamente donde fueron creadas. Esta anonimidad se combina bien con las funciones que veremos a continuación: `map`, `filter`, `reduce`.

## `map`

`map` recibe como parámetros una función y al menos un iterable. Retorna un generador que resulta de aplicar la función sobre cada elemento del iterable. Es así como `map(f, iterable)` es equivalente a `(f(x) for x in iterable)`.

La cantidad de iterables entregada a `map` debe corresponder con la cantidad de parámetros que recible la función `f`. Por ejemplo, si tenemos `map(f, iterable1, iterable2)` entonces `f` debe recibir dos parámetros. Es así como  `map(f, iterable1, iterable2)` es equivalente a `(f(x, y) for x, y in zip(iterable1, iterable2))`.

### Ejemplos

1\. Tenemos una lista de `strings`, donde queremos colocar cada uno en minúsculas:

In [7]:
strings = ['Señores pasajeros', 'Disculpen', 'mi', 'IntencIÓN', 'no', 'Es', 'MolEstar']
mapeo = map(lambda x: x.lower(), strings)

In [8]:
', '.join(mapeo)

'señores pasajeros, disculpen, mi, intención, no, es, molestar'

2\. Tenemos dos o más listas de números y queremos, a partir de esos números, calcular otro:

In [9]:
a = [1, 2, 3, 4]
b = [17, 12, 11, 10]
c = [-1, -4, 5, 9]

mapeo_1 = map(lambda x, y: x ** 2 + y ** 2, a, b)
mapeo_2 = map(lambda x, y, z: x + y ** 2 + z ** 3, a, b, c)

print(list(mapeo_1))
print(list(mapeo_2))

[290, 148, 130, 116]
[289, 82, 249, 833]


Notar que la cantidad de elementos que procesa la función en un `map` corresponde a la cantidad que tiene el iterable más pequeño:

In [10]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
b = [100, 101, 102]

mapeo = map(lambda x, y: x + y, a, b)
print(list(mapeo))

[101, 103, 105]


## `filter`   

`filter(f, iterable)` recibe como parámetros una función que retorna `True` o `False`, y un iterable. Retorna un generador que no entrega elementos donde la función `f` da `False`.

Se puede ver que `filter(f, iterable)` es equivalente a `(x for x in iterable if f(x))`.

In [11]:
def fibonacci(límite):
    a,b = 0,1
    for _ in range(límite):
        yield b
        a, b = b, a + b

filtrado_impares = filter(lambda x: x % 2 != 0, fibonacci(10))
print(list(filtrado_impares))

filtrado_pares = filter(lambda x: x % 2 == 0, fibonacci(10))
print(list(filtrado_pares))

[1, 1, 3, 5, 13, 21, 55]
[2, 8, 34]


Otro ejemplo, en el que se entrega un `set`:

In [12]:
set_filtrado = filter(lambda x: x < 10, {100, 1, 5, 9, 91, 1})
print(list(set_filtrado))

[1, 5, 9]


## `reduce`

Vamos a explicar la idea del `reduce` con un ejemplo de cálculo manual. Imaginemos que tenemos una secuencia con números, y que queremos obtener la suma de ellos. También supongamos que nos complica sumar más de dos números a la vez.

In [13]:
lista = [1, 2, 3, 4, 5, 6]

Si hicieramos esta suma en forma procedural, lo que probablemente haríamos es sumar los dos primeros elementos, guardar el resultado, y ese resultado sumarlo con el siguiente elemento. Y así sucesivamente:

- $1 + 2 = 3$
- $3 + 3 = 6$
- $6 + 4 = 10$
- $10 + 5 = 15$
- $15 + 6 = 21$

El resultado final es **21**. Ahora supongamos que no necesariamente queremos sumar los números de a pares, sino que aplicar una función cualquiera `f`:
- $f(1, 2) = a$
- $f(a, 3) = b$
- $f(b, 4) = c$
- $f(c, 5) = d$
- $f(d, 6) = e$

En este caso, el resultado final es **e**. Reemplazando las variables, nuestro cómputo fue:

$f(f(f(f(f(1, 2), 3), 4), 5), 6)$

Esa es exactamente la idea detrás del `reduce`. Esta operación consiste en aplicar sucesivamente una función `f(x, y)`, donde `x` es el resultado acumulado e `y` es un elemento de la secuencia hasta _reducirla_ a un sólo resultado.

![](1.reduce.png)

Entonces, `reduce(f, [s1, s2, s3, ..., sn])` recibe una función que toma dos valores y una secuencia. Retorna lo que resulta de aplicar la función `f` a la secuencia `[s1, s2, s3, ..., sn]` de la siguiente forma: `f(f(f(f(s1, s2), s3), s4), s5), ...`.

Podemos ver que funciona muy bien para la suma que habíamos propuesto al principio:

In [14]:
from functools import reduce
reduce(lambda x, y: x + y, lista)

21

Y también podemos hacer lo mismo con una función que haga otra cosa más compleja:

In [15]:
reduce(lambda x, y: x ** 2 + y, lista)

480004287

### Ejemplo de aplanamiento de listas

Consideremos que tenemos una lista con más listas dentro, y queremos juntar todos los elementos en orden en una gran lista. Podemos hacer eso con `reduce`.

In [16]:
lista_con_listas = [[1, 2], [3, 4], [5, 6], [7, 8, 9]]
lista_aplanada = reduce(lambda x, y: x + y, lista_con_listas)
print(lista_aplanada)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


### Precauciones

#### Cantidad de elementos de la secuencia a reducir

Cuando la secuencia que se entrega a `reduce` tiene sólo un elemento, la operación retornará sólo ese elemento sin que pase por la función. 

In [17]:
reduce(lambda x, y: x + y, [[1, 2]])

[1, 2]

Podemos entregar un inicializador al `reduce` para que la función se aplique al menos una vez:

In [18]:
reduce(lambda x, y: x + y, [[1, 2]], [0, 0, 0])

[0, 0, 0, 1, 2]

No obstante, si la secuencia entregada es vacía y no entregamos un valor de inicialización, nos da un error:

In [19]:
reduce(lambda x, y: x + y, [])

TypeError: reduce() of empty sequence with no initial value

In [20]:
reduce(lambda x, y: x + y, [], [0, 0, 0])

[0, 0, 0]

#### Reducir _sets_ u otros iterables no ordenados

En este caso hay que tener cuidado cuando la operación que se haga dependa del orden de los elementos, pues el resultado podría no ser el esperado. 

En el ejemplo, se tiene un _set_ con varias palabras que queremos concatenar con un `reduce`. Vemos que el orden final dista del orden que se declaró en el _set_, pues esta estructura no es ordenada.

In [21]:
palabras = {'casa', 'mar', 'ventana', 'roca', 'piso'}

reduce(lambda x, y: "{} {}".format(x, y), palabras)

'ventana piso casa roca mar'